# The Dataset for Pretraining Word Embedding
:label:`sec_word2vec_data`

In this section, we will introduce how to preprocess a dataset with
negative sampling :numref:`sec_approx_train` and load into minibatches for
word2vec training. The dataset we use is [Penn Tree Bank (PTB)]( https://catalog.ldc.upenn.edu/LDC99T42), which is a small but commonly-used corpus. It takes samples from Wall Street Journal articles and includes training sets, validation sets, and test sets.

First, import the packages and modules required for the experiment.


In [1]:
%use @file[../djl.json]
%use lets-plot
@file:DependsOn("../D2J-1.0-SNAPSHOT.jar")
//import jp.live.ugai.d2j.attention.Chap10Utils
import jp.live.ugai.d2j.util.ImageUtils
import jp.live.ugai.d2j.util.StopWatch
import jp.live.ugai.d2j.util.Training
import jp.live.ugai.d2j.timemachine.Vocab

In [2]:
val manager = NDManager.newBaseManager();

## Reading and Preprocessing the Dataset

This dataset has already been preprocessed. Each line of the dataset acts as a sentence. All the words in a sentence are separated by spaces. In the word embedding task, each word is a token.


In [3]:
fun readPTB() : List<List<String>> {
    val ptbURL = "http://d2l-data.s3-accelerate.amazonaws.com/ptb.zip";
    val input = URL(ptbURL).openStream();
    ZipUtils.unzip(input, Paths.get("./"));

    val lines = mutableListOf<String>();
   val file = File("./ptb/ptb.train.txt");
    val myReader = java.util.Scanner(file);
    while (myReader.hasNextLine()) {
        lines.add(myReader.nextLine());
    }
    val tokens = mutableListOf<List<String>>()
    for (i in 0 until lines.size) {
        tokens.add(lines.get(i).trim().split(" "))
    }
    return tokens;
}

In [4]:
val sentences = readPTB();
println("# sentences: " + sentences.size);

# sentences: 42068


Next we build a vocabulary with words appeared not greater than 10 times mapped into a "&lt;unk&gt;" token. Note that the preprocessed PTB data also contains "&lt;unk&gt;" tokens presenting rare words.


In [5]:
val vocab = Vocab(sentences, 10, listOf<String>())
println(vocab.length());

6719


## Subsampling

In text data, there are generally some words that appear at high frequencies, such "the", "a", and "in" in English. Generally speaking, in a context window, it is better to train the word embedding model when a word (such as "chip") and a lower-frequency word (such as "microprocessor") appear at the same time, rather than when a word appears with a higher-frequency word (such as "the"). Therefore, when training the word embedding model, we can perform subsampling on the words :cite:`Mikolov.Sutskever.Chen.ea.2013`. Specifically, each indexed word $w_i$ in the dataset will drop out at a certain probability. The dropout probability is given as:

$$ P(w_i) = \max\left(1 - \sqrt{\frac{t}{f(w_i)}}, 0\right),$$

Here, $f(w_i)$ is the ratio of the instances of word $w_i$ to the total number of words in the dataset, and the constant $t$ is a hyperparameter (set to $10^{-4}$ in this experiment). As we can see, it is only possible to drop out the word $w_i$ in subsampling when $f(w_i) > t$. The higher the word's frequency, the higher its dropout probability.


In [6]:
fun keep(token: String, counter: Map<Any, Double>, numTokens: Int) : Boolean {
    // Return True if to keep this token during subsampling
    return kotlin.random.Random.nextFloat() < Math.sqrt(1e-4 / counter.get(token)!! * numTokens)
}

fun subSampling(sentences: List<List<String>>, vocab: Vocab) : List<List<String>> {
    val tempSentences = mutableListOf<List<String>>()
    for (i in 0 until sentences.size) {
        val tmp = mutableListOf<String>()
        for (j in 0 until sentences[i].size) {
            tmp.add(vocab.idxToToken.get(vocab.getIdx(sentences[i][j])))
        }
        tempSentences.add(tmp)
    }
    // Count the frequency for each word
    val counter = Vocab.countCorpus2D(sentences);
    var numTokens : Int = 0
    for (value in counter.values) {
        numTokens += value
    }

    // Now do the subsampling
    val output = mutableListOf<List<String>>()
    for (i in 0 until tempSentences.size) {
        val tks = mutableListOf<String>()
        for (j in 0 until tempSentences[i].size) {
            val tk = tempSentences[i][j];
            if (keep(tempSentences[i][j], counter.map { it.key to it.value.toDouble() }.toMap() , numTokens)) {
                tks.add(tk);
            }
        }
        output.add(tks)
    }

    return output
}

val subsampled = subSampling(sentences, vocab);

Compare the sequence lengths before and after sampling, we can see subsampling significantly reduced the sequence length.


In [7]:
val y1 = sentences.map { it.size }
val y2 = subsampled.map { it.size }

// HistogramTrace trace1 =
//        HistogramTrace.builder(y1).opacity(.75).name("origin").nBinsX(20).build();
// HistogramTrace trace2 =
//         HistogramTrace.builder(y2).opacity(.75).name("subsampled").nBinsX(20).build();

// Layout layout =
//        Layout.builder()
//                .barMode(Layout.BarMode.GROUP)
//                .showLegend(true)
//                .xAxis(Axis.builder().title("# tokens per sentence").build())
//                .yAxis(Axis.builder().title("count").build())
//                .build();
//new Figure(layout, trace1, trace2);

For individual tokens, the sampling rate of the high-frequency word "the" is less than 1/20.


In [8]:
public static String compareCounts(String token, String[][] sentences, String[][] subsampled) {
    int beforeCount = 0;
    for (int i = 0; i < sentences.length; i++) {
        for (int j = 0; j < sentences[i].length; j++) {
            if (sentences[i][j].equals(token)) beforeCount += 1;
        }
    }

    int afterCount = 0;
    for (int i = 0; i < subsampled.length; i++) {
        for (int j = 0; j < subsampled[i].length; j++) {
            if (subsampled[i][j].equals(token)) afterCount += 1;
        }
    }

    return "# of \"the\": before=" + beforeCount + ", after=" + afterCount;
}

System.out.println(compareCounts("the", sentences, subsampled));

Line_15.jupyter-kts (1:48 - 48) Expecting an element
Line_15.jupyter-kts (1:48 - 49) Expecting an element
Line_15.jupyter-kts (1:57 - 57) Expecting an index element
Line_15.jupyter-kts (1:59 - 59) Expecting an index element
Line_15.jupyter-kts (1:70 - 70) Expecting an element
Line_15.jupyter-kts (1:70 - 71) Expecting an element
Line_15.jupyter-kts (1:79 - 79) Expecting an index element
Line_15.jupyter-kts (1:81 - 81) Expecting an index element
Line_15.jupyter-kts (1:93 - 93) Expecting an element
Line_15.jupyter-kts (1:93 - 94) Expecting an element
Line_15.jupyter-kts (2:21 - 22) Expecting an element
Line_15.jupyter-kts (2:23 - 24) Unexpected tokens (use ';' to separate expressions on the same line)
Line_15.jupyter-kts (2:24 - 25) Expecting an element
Line_15.jupyter-kts (3:14 - 15) Expecting 'in'
Line_15.jupyter-kts (3:15 - 15) Expecting ')'
Line_15.jupyter-kts (3:16 - 16) Expecting an expression
Line_15.jupyter-kts (3:46 - 47) Unexpected tokens (use ';' to separate expressions on the 

But the low-frequency word "join" is completely preserved.


In [9]:
System.out.println(compareCounts("join", sentences, subsampled));

Line_16.jupyter-kts (1:20 - 33) Unresolved reference: compareCounts

Last, we map each token into an index to construct the corpus.


In [10]:
Integer[][] corpus = new Integer[subsampled.length][];
for (int i = 0; i < subsampled.length; i++) {
    corpus[i] = vocab.getIdxs(subsampled[i]);
}
for (int i = 0; i < 3; i++) {
    System.out.println(Arrays.toString(corpus[i]));
}

Line_17.jupyter-kts (1:9 - 9) Expecting an index element
Line_17.jupyter-kts (1:11 - 11) Expecting an index element
Line_17.jupyter-kts (1:20 - 21) Expecting an element
Line_17.jupyter-kts (1:53 - 53) Expecting an index element
Line_17.jupyter-kts (2:10 - 11) Expecting 'in'
Line_17.jupyter-kts (2:11 - 11) Expecting ')'
Line_17.jupyter-kts (2:12 - 12) Expecting an expression
Line_17.jupyter-kts (2:43 - 44) Expecting an element
Line_17.jupyter-kts (5:10 - 11) Expecting 'in'
Line_17.jupyter-kts (5:11 - 11) Expecting ')'
Line_17.jupyter-kts (5:12 - 12) Expecting an expression
Line_17.jupyter-kts (5:27 - 28) Expecting an element

## Loading the Dataset

Next we read the corpus with token indicies into data batches for training.

### Extracting Central Target Words and Context Words

We use words with a distance from the central target word not exceeding the context window size as the context words of the given center target word. The following definition function extracts all the central target words and their context words. It uniformly and randomly samples an integer to be used as the context window size between integer 1 and the `maxWindowSize` (maximum context window).


In [11]:
public static Pair<ArrayList<Integer>, ArrayList<ArrayList<Integer>>> getCentersAndContext(
        Integer[][] corpus, int maxWindowSize) {
    ArrayList<Integer> centers = new ArrayList<>();
    ArrayList<ArrayList<Integer>> contexts = new ArrayList<>();

    for (Integer[] line : corpus) {
        // Each sentence needs at least 2 words to form a "central target word
        // - context word" pair
        if (line.length < 2) {
            continue;
        }
        centers.addAll(Arrays.asList(line));
        for (int i = 0; i < line.length; i++) { // Context window centered at i
            int windowSize = new Random().nextInt(maxWindowSize - 1) + 1;
            List<Integer> indices =
                    IntStream.range(
                                    Math.max(0, i - windowSize),
                                    Math.min(line.length, i + 1 + windowSize))
                            .boxed()
                            .collect(Collectors.toList());
            // Exclude the central target word from the context words
            indices.remove(indices.indexOf(i));
            ArrayList<Integer> context = new ArrayList<>();
            for (Integer idx : indices) {
                context.add(line[idx]);
            }
            contexts.add(context);
        }
    }
    return new Pair<>(centers, contexts);
}

Line_18.jupyter-kts (2:17 - 17) Expecting an index element
Line_18.jupyter-kts (2:19 - 19) Expecting an index element
Line_18.jupyter-kts (2:27 - 27) Expecting an element
Line_18.jupyter-kts (2:27 - 28) Expecting an element
Line_18.jupyter-kts (2:46 - 46) Expecting an element
Line_18.jupyter-kts (2:46 - 47) Expecting an element
Line_18.jupyter-kts (3:32 - 33) Expecting an element
Line_18.jupyter-kts (3:48 - 48) Type expected
Line_18.jupyter-kts (4:44 - 45) Expecting an element
Line_18.jupyter-kts (4:60 - 60) Type expected
Line_18.jupyter-kts (6:17 - 18) Expecting 'in'
Line_18.jupyter-kts (6:18 - 18) Expecting ')'
Line_18.jupyter-kts (6:18 - 34) Unexpected tokens (use ';' to separate expressions on the same line)
Line_18.jupyter-kts (13:18 - 19) Expecting 'in'
Line_18.jupyter-kts (13:19 - 19) Expecting ')'
Line_18.jupyter-kts (13:20 - 20) Expecting an expression
Line_18.jupyter-kts (13:45 - 46) Unexpected tokens (use ';' to separate expressions on the same line)
Line_18.jupyter-kts (14:

Next, we create an artificial dataset containing two sentences of 7 and 3 words, respectively. Assume the maximum context window is 2 and print all the central target words and their context words.


In [12]:
Integer[][] tinyDataset =
        new Integer[][] {
            IntStream.range(0, 7)
                    .boxed()
                    .collect(Collectors.toList())
                    .toArray(new Integer[] {}),
            IntStream.range(7, 10)
                    .boxed()
                    .collect(Collectors.toList())
                    .toArray(new Integer[] {})
        };

System.out.println("dataset " + Arrays.deepToString(tinyDataset));
Pair<ArrayList<Integer>, ArrayList<ArrayList<Integer>>> centerContextPair =
        getCentersAndContext(tinyDataset, 2);
for (int i = 0; i < centerContextPair.getValue().size(); i++) {
    System.out.println(
            "Center "
                    + centerContextPair.getKey().get(i)
                    + " has contexts"
                    + centerContextPair.getValue().get(i));
}

Line_19.jupyter-kts (1:9 - 9) Expecting an index element
Line_19.jupyter-kts (1:11 - 11) Expecting an index element
Line_19.jupyter-kts (1:25 - 26) Expecting an element
Line_19.jupyter-kts (2:23 - 23) Expecting an index element
Line_19.jupyter-kts (6:47 - 48) Unexpected tokens (use ';' to separate expressions on the same line)
Line_19.jupyter-kts (14:75 - 76) Expecting an element
Line_19.jupyter-kts (16:10 - 11) Expecting 'in'
Line_19.jupyter-kts (16:11 - 11) Expecting ')'
Line_19.jupyter-kts (16:12 - 12) Expecting an expression
Line_19.jupyter-kts (16:61 - 62) Expecting an element

We set the maximum context window size to 5. The following extracts all the central target words and their context words in the dataset.


In [13]:
centerContextPair = getCentersAndContext(corpus, 5);
ArrayList<Integer> allCenters = centerContextPair.getKey();
ArrayList<ArrayList<Integer>> allContexts = centerContextPair.getValue();
System.out.println("# center-context pairs:" + allCenters.size());

Line_20.jupyter-kts (2:31 - 32) Expecting an element
Line_20.jupyter-kts (2:50 - 51) Expecting an element
Line_20.jupyter-kts (2:58 - 58) Expecting an expression
Line_20.jupyter-kts (3:43 - 44) Expecting an element
Line_20.jupyter-kts (3:62 - 63) Expecting an element
Line_20.jupyter-kts (3:72 - 72) Expecting an expression

### Negative Sampling

We use negative sampling for approximate training. For a central and context word pair, we randomly sample $K$ noise words ($K=5$ in the experiment). According to the suggestion in the Word2vec paper, the noise word sampling probability $P(w)$ is the ratio of the word frequency of $w$ to the total word frequency raised to the power of 0.75 :cite:`Mikolov.Sutskever.Chen.ea.2013`.

We first define a class to draw a candidate according to the sampling weights. It caches a 10,000 size random number bank.


In [14]:
public class RandomGenerator {
    /* Draw a random int in [0, n] according to n sampling weights. */

    private List<Integer> population;
    private List<Double> samplingWeights;
    private List<Integer> candidates;
    private List<org.apache.commons.math3.util.Pair<Integer, Double>> pmf;
    private int i;

    public RandomGenerator(List<Double> samplingWeights) {
        this.population =
                IntStream.range(0, samplingWeights.size()).boxed().collect(Collectors.toList());
        this.samplingWeights = samplingWeights;
        this.candidates = new ArrayList<>();
        this.i = 0;

        this.pmf = new ArrayList<>();
        for (int i = 0; i < samplingWeights.size(); i++) {
            this.pmf.add(new org.apache.commons.math3.util.Pair(this.population.get(i), this.samplingWeights.get(i).doubleValue()));
        }
    }

    public Integer draw() {
        if (this.i == this.candidates.size()) {
            this.candidates =
                    Arrays.asList((Integer[]) new EnumeratedDistribution(this.pmf).sample(10000, new Integer[] {}));
            this.i = 0;
        }
        this.i += 1;
        return this.candidates.get(this.i - 1);
    }
}

RandomGenerator generator =
        new RandomGenerator(Arrays.asList(new Double[] {2.0, 3.0, 4.0}));
Integer[] generatorOutput = new Integer[10];
for (int i = 0; i < 10; i++) {
    generatorOutput[i] = generator.draw();
}
System.out.println(Arrays.toString(generatorOutput));

Line_21.jupyter-kts (4:13 - 17) Expecting member declaration
Line_21.jupyter-kts (4:17 - 18) Expecting member declaration
Line_21.jupyter-kts (4:18 - 25) Expecting member declaration
Line_21.jupyter-kts (4:25 - 26) Expecting member declaration
Line_21.jupyter-kts (4:27 - 37) Expecting member declaration
Line_21.jupyter-kts (5:13 - 17) Expecting member declaration
Line_21.jupyter-kts (5:17 - 18) Expecting member declaration
Line_21.jupyter-kts (5:18 - 24) Expecting member declaration
Line_21.jupyter-kts (5:24 - 25) Expecting member declaration
Line_21.jupyter-kts (5:26 - 41) Expecting member declaration
Line_21.jupyter-kts (6:13 - 17) Expecting member declaration
Line_21.jupyter-kts (6:17 - 18) Expecting member declaration
Line_21.jupyter-kts (6:18 - 25) Expecting member declaration
Line_21.jupyter-kts (6:25 - 26) Expecting member declaration
Line_21.jupyter-kts (6:27 - 37) Expecting member declaration
Line_21.jupyter-kts (7:13 - 17) Expecting member declaration
Line_21.jupyter-kts (7:1

In [15]:
public static ArrayList<ArrayList<Integer>> getNegatives(
        ArrayList<ArrayList<Integer>> allContexts, Integer[][] corpus, int K) {
    LinkedHashMap<?, Integer> counter = Vocab.countCorpus2D(corpus);
    ArrayList<Double> samplingWeights = new ArrayList<>();
    for (Map.Entry<?, Integer> entry : counter.entrySet()) {
        samplingWeights.add(Math.pow(entry.getValue(), .75));
    }
    ArrayList<ArrayList<Integer>> allNegatives = new ArrayList<>();
    RandomGenerator generator = new RandomGenerator(samplingWeights);
    for (ArrayList<Integer> contexts : allContexts) {
        ArrayList<Integer> negatives = new ArrayList<>();
        while (negatives.size() < contexts.size() * K) {
            Integer neg = generator.draw();
            // Noise words cannot be context words
            if (!contexts.contains(neg)) {
                negatives.add(neg);
            }
        }
        allNegatives.add(negatives);
    }
    return allNegatives;
}

ArrayList<ArrayList<Integer>> allNegatives = getNegatives(allContexts, corpus, 5);

Line_22.jupyter-kts (2:50 - 50) Expecting an element
Line_22.jupyter-kts (2:50 - 51) Expecting an element
Line_22.jupyter-kts (2:60 - 60) Expecting an index element
Line_22.jupyter-kts (2:62 - 62) Expecting an index element
Line_22.jupyter-kts (2:70 - 70) Expecting an element
Line_22.jupyter-kts (2:70 - 71) Expecting an element
Line_22.jupyter-kts (2:77 - 77) Expecting an element
Line_22.jupyter-kts (2:77 - 78) Expecting an element
Line_22.jupyter-kts (3:19 - 20) Type expected
Line_22.jupyter-kts (3:39 - 40) Expecting an element
Line_22.jupyter-kts (3:46 - 47) Expecting an element
Line_22.jupyter-kts (4:39 - 40) Expecting an element
Line_22.jupyter-kts (4:55 - 55) Type expected
Line_22.jupyter-kts (5:13 - 14) Expecting 'in'
Line_22.jupyter-kts (5:14 - 14) Expecting ')'
Line_22.jupyter-kts (5:20 - 21) Type expected
Line_22.jupyter-kts (5:38 - 39) Expecting an element
Line_22.jupyter-kts (5:47 - 48) Expecting an element
Line_22.jupyter-kts (5:57 - 57) Expecting an expression
Line_22.jupy

### Reading into Batches

We extract all central target words `allCenters`, and the context words `allContexts` and noise words `allNegatives` of each central target word from the dataset. We will read them in random minibatches.

In a minibatch of data, the $i^\mathrm{th}$ example includes a central word and its corresponding $n_i$ context words and $m_i$ noise words. Since the context window size of each example may be different, the sum of context words and noise words, $n_i+m_i$, will be different. When constructing a minibatch, we concatenate the context words and noise words of each example, and add 0s for padding until the length of the concatenations are the same, that is, the length of all concatenations is $\max_i n_i+m_i$(`maxLen`). In order to avoid the effect of padding on the loss function calculation, we construct the mask variable `masks`, each element of which corresponds to an element in the concatenation of context and noise words, `contextsNegatives`. When an element in the variable `contextsNegatives` is a padding, the element in the mask variable `masks` at the same position will be 0. Otherwise, it takes the value 1. In order to distinguish between positive and negative examples, we also need to distinguish the context words from the noise words in the `contextsNegatives` variable. Based on the construction of the mask variable, we only need to create a label variable `labels` with the same shape as the `contextsNegatives` variable and set the elements corresponding to context words (positive examples) to 1, and the rest to 0.

Next, we will implement the minibatch reading function `batchifyData`. Its minibatch input `data` is a list of `NDArrays`, each element of which contains central target words `center`, context words `context`, and noise words `negative`. The minibatch data returned by this function conforms to the format we need, for example, it includes the mask variable.


In [16]:
public static NDList batchifyData(NDList[] data) {
    NDList centers = new NDList();
    NDList contextsNegatives = new NDList();
    NDList masks = new NDList();
    NDList labels = new NDList();

    long maxLen = 0;
    for (NDList ndList : data) { // center, context, negative = ndList
        maxLen =
                Math.max(
                        maxLen,
                        ndList.get(1).countNonzero().getLong()
                                + ndList.get(2).countNonzero().getLong());
    }
    for (NDList ndList : data) { // center, context, negative = ndList
        NDArray center = ndList.get(0);
        NDArray context = ndList.get(1);
        NDArray negative = ndList.get(2);

        int count = 0;
        for (int i = 0; i < context.size(); i++) {
            // If a 0 is found, we want to stop adding these
            // values to NDArray
            if (context.get(i).getInt() == 0) {
                break;
            }
            contextsNegatives.add(context.get(i).reshape(1));
            masks.add(manager.create(1).reshape(1));
            labels.add(manager.create(1).reshape(1));
            count += 1;
        }
        for (int i = 0; i < negative.size(); i++) {
            // If a 0 is found, we want to stop adding these
            // values to NDArray
            if (negative.get(i).getInt() == 0) {
                break;
            }
            contextsNegatives.add(negative.get(i).reshape(1));
            masks.add(manager.create(1).reshape(1));
            labels.add(manager.create(0).reshape(1));
            count += 1;
        }
        // Fill with zeroes remaining array
        while (count != maxLen) {
            contextsNegatives.add(manager.create(0).reshape(1));
            masks.add(manager.create(0).reshape(1));
            labels.add(manager.create(0).reshape(1));
            count += 1;
        }

        // Add this NDArrays to output NDArrays
        centers.add(center.reshape(1));
    }
    return new NDList(
            NDArrays.concat(centers).reshape(data.length, -1),
            NDArrays.concat(contextsNegatives).reshape(data.length, -1),
            NDArrays.concat(masks).reshape(data.length, -1),
            NDArrays.concat(labels).reshape(data.length, -1));
}

Line_23.jupyter-kts (1:42 - 42) Expecting an index element
Line_23.jupyter-kts (1:48 - 48) Expecting an element
Line_23.jupyter-kts (2:20 - 21) Expecting an element
Line_23.jupyter-kts (3:30 - 31) Expecting an element
Line_23.jupyter-kts (4:18 - 19) Expecting an element
Line_23.jupyter-kts (5:19 - 20) Expecting an element
Line_23.jupyter-kts (7:17 - 18) Expecting an element
Line_23.jupyter-kts (7:19 - 20) Unexpected tokens (use ';' to separate expressions on the same line)
Line_23.jupyter-kts (7:20 - 21) Expecting an element
Line_23.jupyter-kts (8:17 - 23) Expecting 'in'
Line_23.jupyter-kts (8:23 - 23) Expecting ')'
Line_23.jupyter-kts (8:24 - 24) Expecting an expression
Line_23.jupyter-kts (8:24 - 31) Unexpected tokens (use ';' to separate expressions on the same line)
Line_23.jupyter-kts (15:17 - 23) Expecting 'in'
Line_23.jupyter-kts (15:23 - 23) Expecting ')'
Line_23.jupyter-kts (15:24 - 24) Expecting an expression
Line_23.jupyter-kts (15:24 - 31) Unexpected tokens (use ';' to sepa

Construct two simple examples:


In [17]:
NDList x1 =
        new NDList(
                manager.create(new int[] {1}),
                manager.create(new int[] {2, 2}),
                manager.create(new int[] {3, 3, 3, 3}));
NDList x2 =
        new NDList(
                manager.create(new int[] {1}),
                manager.create(new int[] {2, 2, 2}),
                manager.create(new int[] {3, 3}));

NDList batchedData = batchifyData(new NDList[] {x1, x2});
String[] names = new String[] {"centers", "contexts_negatives", "masks", "labels"};
for (int i = 0; i < batchedData.size(); i++) {
    System.out.println(names[i] + " shape: " + batchedData.get(i));
}

Line_24.jupyter-kts (1:11 - 12) Expecting an element
Line_24.jupyter-kts (3:46 - 46) Expecting ')'
Line_24.jupyter-kts (3:46 - 47) Expecting an element
Line_24.jupyter-kts (4:44 - 47) Unexpected tokens (use ';' to separate expressions on the same line)
Line_24.jupyter-kts (4:49 - 50) Expecting an element
Line_24.jupyter-kts (5:44 - 53) Unexpected tokens (use ';' to separate expressions on the same line)
Line_24.jupyter-kts (5:55 - 56) Expecting an element
Line_24.jupyter-kts (6:11 - 12) Expecting an element
Line_24.jupyter-kts (8:46 - 46) Expecting ')'
Line_24.jupyter-kts (8:46 - 47) Expecting an element
Line_24.jupyter-kts (9:44 - 50) Unexpected tokens (use ';' to separate expressions on the same line)
Line_24.jupyter-kts (9:52 - 53) Expecting an element
Line_24.jupyter-kts (10:44 - 47) Unexpected tokens (use ';' to separate expressions on the same line)
Line_24.jupyter-kts (10:49 - 50) Expecting an element
Line_24.jupyter-kts (12:20 - 21) Expecting an element
Line_24.jupyter-kts (12:

We use the `batchifyData` function just defined to specify the minibatch reading method for the `ArrayDataset` instance iterator.

## Putting All Things Together

Last, we define the `loadDataPTB` function that read the PTB dataset and return the dataset. In addition, we will create a function called `convertNDArray` that will convert the `centers`, `contexts`, and `negatives` lists, into `NDArrays` by putting 0s where there is no data in order for the rows to have the same lenghts.


In [18]:
public static NDList convertNDArray(Object[] data, NDManager manager) {
    ArrayList<Integer> centers = (ArrayList<Integer>) data[0];
    ArrayList<ArrayList<Integer>> contexts = (ArrayList<ArrayList<Integer>>) data[1];
    ArrayList<ArrayList<Integer>> negatives = (ArrayList<ArrayList<Integer>>) data[2];

    // Create centers NDArray
    NDArray centersNDArray = manager.create(centers.stream().mapToInt(i -> i).toArray());

    // Create contexts NDArray
    int maxLen = 0;
    for (ArrayList<Integer> context : contexts) {
        maxLen = Math.max(maxLen, context.size());
    }
    // Fill arrays with 0s to all have same lengths and be able to create NDArray
    for (ArrayList<Integer> context : contexts) {
        while (context.size() != maxLen) {
            context.add(0);
        }
    }
    NDArray contextsNDArray =
            manager.create(
                    contexts.stream()
                            .map(u -> u.stream().mapToInt(i -> i).toArray())
                            .toArray(int[][]::new));

    // Create negatives NDArray
    maxLen = 0;
    for (ArrayList<Integer> negative : negatives) {
        maxLen = Math.max(maxLen, negative.size());
    }
    // Fill arrays with 0s to all have same lengths and be able to create NDArray
    for (ArrayList<Integer> negative : negatives) {
        while (negative.size() != maxLen) {
            negative.add(0);
        }
    }
    NDArray negativesNDArray =
            manager.create(
                    negatives.stream()
                            .map(u -> u.stream().mapToInt(i -> i).toArray())
                            .toArray(int[][]::new));

    return new NDList(centersNDArray, contextsNDArray, negativesNDArray);
}

public static Pair<ArrayDataset, Vocab> loadDataPTB(
        int batchSize, int maxWindowSize, int numNoiseWords, NDManager manager)
        throws IOException, TranslateException {
    String[][] sentences = readPTB();
    Vocab vocab = new Vocab(sentences, 10, new String[] {});
    String[][] subSampled = subSampling(sentences, vocab);
    Integer[][] corpus = new Integer[subSampled.length][];
    for (int i = 0; i < subSampled.length; i++) {
        corpus[i] = vocab.getIdxs(subSampled[i]);
    }
    Pair<ArrayList<Integer>, ArrayList<ArrayList<Integer>>> pair =
            getCentersAndContext(corpus, maxWindowSize);
    ArrayList<ArrayList<Integer>> negatives =
            getNegatives(pair.getValue(), corpus, numNoiseWords);

    NDList ndArrays =
            convertNDArray(new Object[] {pair.getKey(), pair.getValue(), negatives}, manager);
    ArrayDataset dataset =
            new ArrayDataset.Builder()
                    .setData(ndArrays.get(0), ndArrays.get(1), ndArrays.get(2))
                    .optDataBatchifier(
                            new Batchifier() {
                                @Override
                                public NDList batchify(NDList[] ndLists) {
                                    return batchifyData(ndLists);
                                }

                                @Override
                                public NDList[] unbatchify(NDList ndList) {
                                    return new NDList[0];
                                }
                            })
                    .setSampling(batchSize, true)
                    .build();

    return new Pair<>(dataset, vocab);
}

Line_25.jupyter-kts (1:44 - 44) Expecting an index element
Line_25.jupyter-kts (1:50 - 50) Expecting an element
Line_25.jupyter-kts (1:50 - 51) Expecting an element
Line_25.jupyter-kts (1:69 - 69) Expecting an element
Line_25.jupyter-kts (1:69 - 70) Expecting an element
Line_25.jupyter-kts (2:32 - 33) Expecting an element
Line_25.jupyter-kts (3:44 - 45) Expecting an element
Line_25.jupyter-kts (4:45 - 46) Expecting an element
Line_25.jupyter-kts (7:28 - 29) Expecting an element
Line_25.jupyter-kts (7:37 - 38) Expecting an element
Line_25.jupyter-kts (7:72 - 72) Expecting ')'
Line_25.jupyter-kts (7:73 - 89) Unexpected tokens (use ';' to separate expressions on the same line)
Line_25.jupyter-kts (7:89 - 90) Expecting an element
Line_25.jupyter-kts (10:16 - 17) Expecting an element
Line_25.jupyter-kts (10:18 - 19) Unexpected tokens (use ';' to separate expressions on the same line)
Line_25.jupyter-kts (10:19 - 20) Expecting an element
Line_25.jupyter-kts (11:19 - 20) Expecting 'in'
Line_2

Let us print the first minibatch of the dataset.


In [19]:
Pair<ArrayDataset, Vocab> datasetVocab = loadDataPTB(512, 5, 5, manager);
ArrayDataset dataset = datasetVocab.getKey();
vocab = datasetVocab.getValue();

Batch batch = dataset.getData(manager).iterator().next();
for (int i = 0; i < batch.getData().size(); i++) {
    System.out.println(names[i] + " shape: " + batch.getData().get(i).getShape());
}

Line_26.jupyter-kts (1:40 - 41) Expecting an element
Line_26.jupyter-kts (1:57 - 57) Expecting ')'
Line_26.jupyter-kts (1:57 - 58) Expecting an element
Line_26.jupyter-kts (1:60 - 61) Expecting an element
Line_26.jupyter-kts (1:63 - 64) Expecting an element
Line_26.jupyter-kts (1:72 - 73) Expecting an element
Line_26.jupyter-kts (2:22 - 23) Expecting an element
Line_26.jupyter-kts (2:36 - 37) Expecting an element
Line_26.jupyter-kts (2:44 - 44) Expecting an expression
Line_26.jupyter-kts (5:13 - 14) Expecting an element
Line_26.jupyter-kts (5:22 - 23) Expecting an element
Line_26.jupyter-kts (6:10 - 11) Expecting 'in'
Line_26.jupyter-kts (6:11 - 11) Expecting ')'
Line_26.jupyter-kts (6:12 - 12) Expecting an expression
Line_26.jupyter-kts (6:48 - 49) Expecting an element

## Summary

* Subsampling attempts to minimize the impact of high-frequency words on the training of a word embedding model.
* We can pad examples of different lengths to create minibatches with examples of all the same length and use mask variables to distinguish between padding and non-padding elements, so that only non-padding elements participate in the calculation of the loss function.

## Exercises

1. We use the `batchifyData` function to specify the minibatch reading method for the `ArrayDataset` instance iterator and print the shape of each variable in the first batch read. How should these shapes be calculated?
